In [6]:
from sqlalchemy import create_engine

template_engine = "postgresql+psycopg://{db_username}:{db_password}@{db_host}:{db_port}"

engine_config = template_engine.format(
        db_username="a",
        db_password="b",
        db_host="c.cosqamqjez6h.ap-northeast-2.rds.amazonaws.com",
        db_port="5432",
        db_name="d")

engine = create_engine(engine_config,
    echo=True
)

### Method
`engine.connect()` + `connection.commmit()`

In [9]:

from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    #conn.commit()

2023-03-05 15:48:01,842 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-05 15:48:01,844 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-03-05 15:48:01,846 INFO sqlalchemy.engine.Engine [generated in 0.00374s] {}
2023-03-05 15:48:01,881 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2023-03-05 15:48:01,882 INFO sqlalchemy.engine.Engine [generated in 0.00130s] [{'x': 1, 'y': 1}, {'x': 2, 'y': 4}]
2023-03-05 15:48:01,909 INFO sqlalchemy.engine.Engine ROLLBACK


### Method Begin Once
`engine.begin()` (this includes commit at the end0

- There is also another style of committing data, which is that we can declare our “connect” block to be a transaction block up front. 
- For this mode of operation, we use the Engine.begin() method to acquire the connection, rather than the Engine.connect() method. 
- This method will both manage the scope of the Connection and also enclose everything inside of a transaction with COMMIT at the end, assuming a successful block, or ROLLBACK in case of exception raise. 
- **“Begin once” style is often preferred as it is more succinct and indicates the intention of the entire block up front.**

In [10]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-03-05 15:48:06,203 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-05 15:48:06,205 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2023-03-05 15:48:06,206 INFO sqlalchemy.engine.Engine [cached since 4.325s ago] [{'x': 6, 'y': 8}, {'x': 9, 'y': 10}]
2023-03-05 15:48:06,241 INFO sqlalchemy.engine.Engine ROLLBACK


ProgrammingError: (psycopg.errors.UndefinedTable) relation "some_table" does not exist
LINE 1: INSERT INTO some_table (x, y) VALUES ($1, $2)
                    ^
[SQL: INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)]
[parameters: [{'x': 6, 'y': 8}, {'x': 9, 'y': 10}]]
(Background on this error at: https://sqlalche.me/e/20/f405)